In [14]:
import numpy as np
data = np.load("exercise1.npz")
XA1 = data['XA1']
yA1 = data['yA1']
XA2 = data['XA2']
yA2 = data['yA2']

XB1 = data['XB1']
yB1 = data['yB1']
XB2 = data['XB2']
yB2 = data['yB2']
data.close()

In [123]:
from sklearn import linear_model
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score


def evaluate_performance(X,y,cv_):
    reg = linear_model.LinearRegression()
    cv_results = cross_validate(reg, X, y, cv=cv_, scoring=('r2','neg_mean_squared_error'))
    sorted(cv_results.keys())
    #print(cv_results)
    ['fit_time', 'score_time', 'test_score']
    test_neg_mean_squared_error = np.mean(cv_results['test_neg_mean_squared_error'])
    test_r2 = np.mean(cv_results['test_r2'])
    return test_neg_mean_squared_error, test_r2



In [81]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np

def random_forest_feature_selection(X, y, nr_ests = 10):
    rf = RandomForestRegressor(n_estimators = nr_ests, max_depth = 10, min_samples_leaf = 5)
    rf.fit(X, y)
    feature_importance = rf.feature_importances_
    sorted_features = sorted(range(len(feature_importance)), key=lambda k: feature_importance[k])
    return sorted_features

In [ ]:
xData = XA1
yData = yA1 

nrFeatures = xData.shape[1]

performance_random_forest = []
random_forest_features = random_forest_feature_selection(xData, yData, nr_ests = 100)
for i in range(1,nrFeatures):
    features_i = random_forest_features[:i]
   # print(features_i)
   # print(xData[:,features_i])
   # print(evaluate_performance(xData[features_i,:],yData,10))
    performance_i = (evaluate_performance(xData[:,features_i],yData,10)
    performance_random_forest.append(performance_i[0])
performance_random_forest
    
    

In [139]:
import matplotlib.pyplot as plt
print(performance_random_forest[:,0])
plt.plot(performance_random_forest[:,0], range(1,nrFeatures))
plt.ylabel('some numbers')
plt.show()

TypeError: list indices must be integers or slices, not tuple

In [143]:
print(performance_random_forest.tolist())

AttributeError: 'list' object has no attribute 'tolist'